# Settings

In [1]:
DATASET = "../data/raw/enveda_library_subset_10percent.parquet"
BASE_MODEL = "seyonec/ChemBERTa-zinc-base-v1"
MAX_FRAGMENTS = 512 # from anton, max number of mzs/intensities
MAX_SEQ_LENGTH = 512 # base model max seq length

# Messing around with ChemBERTa for fun and for education

The first half of this colab is just fun experiments trying to understand ChemBERTa and it's tokenizer better.

In [2]:
from transformers import (AutoModelForMaskedLM, AutoTokenizer)

model = AutoModelForMaskedLM.from_pretrained(BASE_MODEL)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Some weights of the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Data prep


These are the columns in the data set:

precursor_mz - f64
precursor_charge - f64
mzs - list[f64]
intensities - list[f64]
in_silico - bool
smiles - str
adduct - str
collision_energy - str
instrument_type - str
compound_class - str
entropy - f64
scaffold_smiles - str

In [3]:
# import the data (with pandas?)
import pandas as pd
import polars as pl

## Load the dataset (for some reason this didn't work for me)
# df = pd.read_parquet('enveda_library_subset 2.parquet')

# print(df.head())


# custom Dataset class for all the types of data.
# I think we might want to make a new 'column' of data that combines mzs and intensities into "label"

from src.team5.data.data_loader import SMILESDataset
from src.team5.data.data_split import sort_dataframe_by_scaffold, split_dataframe
from src.team5.data.prepare import tensorize

df = pl.read_parquet(DATASET)

df_sorted = sort_dataframe_by_scaffold(df)

df_train, df_test = split_dataframe(df_sorted, split_ratio=0.9)

# Print column names
print(df_train.columns)
print(df_test.columns)
print(df_train.head())
print(df_test.head())

(train_tokenized_smiles, train_attention_mask, train_labels, train_supplementary_data) = tensorize(df_train)
(test_tokenized_smiles, test_attention_mask, test_labels, test_supplementary_data) = tensorize(df_test)

train_dataset = SMILESDataset(train_tokenized_smiles, train_attention_mask, train_labels, train_supplementary_data)
test_dataset = SMILESDataset(test_tokenized_smiles, test_attention_mask, test_labels, test_supplementary_data)

## batch?

['precursor_mz', 'precursor_charge', 'mzs', 'intensities', 'in_silico', 'smiles', 'adduct', 'collision_energy', 'instrument_type', 'compound_class', 'entropy', 'scaffold_smiles']
['precursor_mz', 'precursor_charge', 'mzs', 'intensities', 'in_silico', 'smiles', 'adduct', 'collision_energy', 'instrument_type', 'compound_class', 'entropy', 'scaffold_smiles']
shape: (5, 12)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ precursor ┆ precursor ┆ mzs       ┆ intensiti ┆ … ┆ instrumen ┆ compound_ ┆ entropy  ┆ scaffold_ │
│ _mz       ┆ _charge   ┆ ---       ┆ es        ┆   ┆ t_type    ┆ class     ┆ ---      ┆ smiles    │
│ ---       ┆ ---       ┆ list[f64] ┆ ---       ┆   ┆ ---       ┆ ---       ┆ f64      ┆ ---       │
│ f64       ┆ f64       ┆           ┆ list[f64] ┆   ┆ str       ┆ str       ┆          ┆ str       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ 657.54634 ┆ -1.0   

In [4]:
print({k:v.dtype for k,v in train_dataset[0].items()})


{'input_ids': torch.int64, 'attention_mask': torch.int64, 'labels': torch.float32, 'supplementary_data': torch.float32}


# Custom model for our problem
This is probably the most important part in terms of design choices. We are changing the ChemBERTa model by adding on something at the end. This new module will take the hidden SMILES embedding from the last hidden layer as input. It will also take in all the other data about the precusor molecule and experimental conditions (eg, precusor mz, collison energy etc). For now, let's call that supplementary data.

I've written the simplest possible thing here: a single linear layer that takes the embedding of the entire seq, concatinated with all the supplementary data for the example. It outputs "labels", which is mzs and intensities zipped together.

The reason for making a single module output both mzs and intensities is because there needs to be the same number of fragments per example, and the two numbers are very related.

A single linear layer is probably a terrible choice though, since this is the only layer that sees all the supplementary data.

In [5]:
from src.team5.models.custom_model import CustomChemBERTaModel

MS_model = CustomChemBERTaModel(model, MAX_FRAGMENTS, MAX_SEQ_LENGTH)

print(MS_model)

for name, param in MS_model.named_parameters():
    if param.requires_grad:
        print(f"{name} has shape {param.shape}")

CustomChemBERTaModel(
  (model): RobertaForMaskedLM(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(767, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear(

# LoRA config


In [6]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "value"],
    modules_to_save=[
        "final_layers"
    ],  # change this to the name of the new modules at the end.
    bias="none",
)

peft_model = get_peft_model(MS_model, peft_config)

peft_model.print_trainable_parameters()  # check that it's training the right things
for name, param in peft_model.named_parameters():
    if param.requires_grad:
        print(f"{name} is trainable")

trainable params: 2,324,800 || all params: 48,608,383 || trainable%: 4.7827
base_model.model.model.roberta.encoder.layer.0.attention.self.query.lora_A.default.weight is trainable
base_model.model.model.roberta.encoder.layer.0.attention.self.query.lora_B.default.weight is trainable
base_model.model.model.roberta.encoder.layer.0.attention.self.value.lora_A.default.weight is trainable
base_model.model.model.roberta.encoder.layer.0.attention.self.value.lora_B.default.weight is trainable
base_model.model.model.roberta.encoder.layer.1.attention.self.query.lora_A.default.weight is trainable
base_model.model.model.roberta.encoder.layer.1.attention.self.query.lora_B.default.weight is trainable
base_model.model.model.roberta.encoder.layer.1.attention.self.value.lora_A.default.weight is trainable
base_model.model.model.roberta.encoder.layer.1.attention.self.value.lora_B.default.weight is trainable
base_model.model.model.roberta.encoder.layer.2.attention.self.query.lora_A.default.weight is trainab

# Training the Model

In [7]:
from transformers import TrainingArguments
import torch
from transformers import Trainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
peft_model.to(device)

training_args = TrainingArguments(
    output_dir="../logs/test_trainer",
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=0.1,
    label_names=["labels"],
    report_to="none",
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

/Users/lars/workspace/scratch_repository/.venv/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/35055 [00:00<?, ?it/s]

RuntimeError: a Tensor with 4096 elements cannot be converted to Scalar

# Inference

In [ ]:
def prepare_inference_input(smiles, precursor_mz):
    inputs = tokenizer(
        smiles,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt",
    )
    inputs = {key: val.to(device) for key, val in inputs.items()}
    inputs["supplementary_data"] = torch.tensor(
        [supplementary_data], dtype=torch.float
    ).to(device)
    return inputs


peft_model.eval()

# Example data
smiles_example = "CCO"
supplementary_data_example = 0  # TODO

# Prepare input
inputs = prepare_inference_input(smiles_example, supplementary_data_example)

# Inference
with torch.no_grad():
    outputs = peft_model(**inputs)
    logits = outputs[0]

# Choices that affect the whole architecture

*   Format for the supplementary data
*   Format for the label data
*   The format of the output of the new model



### More modular choices (that are important)


*   Whether we have to predict compound_class at inference
*   Include in_silico data?
*   Architeture of the modified ChemBERTa model
*   LoRA parameters


